In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Labels (Kaggle ground truth)
labels = pd.read_csv(
    "F:/AI Project/churn-retention-platform/data/raw/train.csv",
    usecols=["msno", "is_churn"],
    dtype={"msno": "category", "is_churn": "int8"}
)

# Transactions
transactions = pd.read_csv(
    "F:/AI Project/churn-retention-platform/data/raw/transactions.csv",
    usecols=[
        "msno",
        "transaction_date",
        "actual_amount_paid",
        "payment_plan_days",
        "is_cancel"
    ],
    dtype={
        "msno": "category",
        "actual_amount_paid": "float32",
        "payment_plan_days": "int16",
        "is_cancel": "int8"
    }
)

transactions["transaction_date"] = pd.to_datetime(
    transactions["transaction_date"],
    format="%Y%m%d",
    cache=True
)

# Tring to find a more T systematically but can't go much back farther since there would be much more noise
T_candidates = [0.50, 0.55, 0.60, 0.65, 0.70]

# 
diagnostics = []

for q in T_candidates:
    T = transactions["transaction_date"].quantile(q)

    feature_start = T - pd.Timedelta(days=60)
    feature_end   = T

    tx_feat = transactions[
        (transactions["transaction_date"] >= feature_start) &
        (transactions["transaction_date"] <  feature_end)
    ]

    # Aggregate ONLY num_transactions (cheap)
    txn_counts = (
        tx_feat
        .groupby("msno", observed=True)["transaction_date"]
        .count()
        .rename("num_transactions")
        .reset_index()
    )

    temp = labels.merge(txn_counts, on="msno", how="left")
    temp["num_transactions"].fillna(0, inplace=True)

    med = temp.groupby("is_churn")["num_transactions"].median()
    pct_zero = (
        (temp[(temp.is_churn == 1) & (temp.num_transactions == 0)].shape[0]) /
        (temp[temp.is_churn == 1].shape[0])
    )

    diagnostics.append({
        "T_quantile": q,
        "median_churn": med.loc[1],
        "median_nonchurn": med.loc[0],
        "pct_zero_churn": pct_zero
    })

    # IMPORTANT: delete temporary objects
    del tx_feat, txn_counts, temp

diag_df = pd.DataFrame(diagnostics)
diag_df


C:\Users\ankit\AppData\Local\Temp\ipykernel_2884\2089202467.py:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temp["num_transactions"].fillna(0, inplace=True)
C:\Users\ankit\AppData\Local\Temp\ipykernel_2884\2089202467.py:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

,T_quantile,median_churn,median_nonchurn,pct_zero_churn
0,0.50,1.0,2.0,0.498779
1,0.55,0.0,2.0,0.566558
2,0.60,0.0,2.0,0.563202
3,0.65,0.0,2.0,0.535662
4,0.70,1.0,2.0,0.428337


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Labels (Kaggle ground truth)
labels = pd.read_csv(
    "F:/AI Project/churn-retention-platform/data/raw/train.csv",
    usecols=["msno", "is_churn"],
    dtype={"msno": "category", "is_churn": "int8"}
)

# Transactions
transactions = pd.read_csv(
    "F:/AI Project/churn-retention-platform/data/raw/transactions.csv",
    usecols=[
        "msno",
        "transaction_date",
        "actual_amount_paid",
        "payment_plan_days",
        "is_cancel"
    ],
    dtype={
        "msno": "category",
        "actual_amount_paid": "float32",
        "payment_plan_days": "int16",
        "is_cancel": "int8"
    }
)

transactions["transaction_date"] = pd.to_datetime(
    transactions["transaction_date"],
    format="%Y%m%d",
    cache=True
)

FINAL_T = transactions["transaction_date"].quantile(0.70)

feature_start = FINAL_T - pd.Timedelta(days=60)
feature_end   = FINAL_T

tx_feat = transactions[
    (transactions["transaction_date"] >= feature_start) &
    (transactions["transaction_date"] <  feature_end)
]

features = (
    tx_feat
    .groupby("msno", observed=True)
    .agg(
        total_amount_paid=("actual_amount_paid", "sum"),
        num_transactions=("transaction_date", "count"),
        num_cancellations=("is_cancel", "sum"),
        total_plan_days=("payment_plan_days", "sum")
    )
    .reset_index()
)

dataset = labels.merge(features, on="msno", how="left")
dataset.fillna(0, inplace=True)

differenceChurnWise=dataset.groupby("is_churn")[
    ["total_amount_paid", "num_transactions", "num_cancellations","total_plan_days"]
].mean()

X = dataset.drop(columns=["msno", "is_churn"])
y = dataset["is_churn"]

X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)
model.fit(X_tr, y_tr)

y_proba = model.predict_proba(X_val)[:, 1]
ROC_Score=roc_auc_score(y_val, y_proba)

print(f"Roc_score is:{ROC_Score}\n\n & Values are:\n\n{differenceChurnWise}")



Roc_score is:0.6925032226901902

 & Values are:

          total_amount_paid  num_transactions  num_cancellations  \
is_churn                                                           
0                200.103745          1.536332           0.019523   
1                165.557724          0.992690           0.014479   

          total_plan_days  
is_churn                   
0               46.792578  
1               37.636795  
